In [10]:
# Packages
from pathlib import Path
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import re

In [11]:
root = Path('TS')                       # folder where files are stored
features = []                           # list which will store all features
for f in root.glob('*.h5'):             # iterate over all files
    df = pd.read_hdf(f,'df')            # read data
    df.set_index(["station","time"], drop=True, inplace=True) # set index so that pandas can easily merge files
    feature = df["feature"]             # extract only the feature column
    feature.name=re.sub(r'(ERA5_hourly_data_on_pressure_levels_|ERA5_hourly_data_on_single_levels_specific_|ERA5_hourly_data_on_single_levels_)','',f.stem)
    # feature.name = f.stem.replace('ERA5_hourly_data_on_pressure_levels_','')
    # feature.name = f.stem.replace('ERA5_hourly_data_on_single_levels_specific_','') # rename feature to shorten names
    # feature.name = f.stem.replace('ERA5_hourly_data_on_single_levels_','')
    features.append(feature)            # store all features in a list
Xy = pd.concat(features, axis=1)        # concatenate all features into one matrix
Xy = Xy.dropna(how="any", axis=0)       # drop some NAN stations 


In [12]:
Xy['doy']=Xy.index.get_level_values(1).dayofyear
Xy['hod']=Xy.index.get_level_values(1).hour
Xy['t']=[dt.timestamp() for dt in Xy.index.get_level_values(1).to_pydatetime()]
Xy['sin_lon']=np.sin(Xy['Longitude']*np.pi/180)
Xy['cos_lon']=np.cos(Xy['Longitude']*np.pi/180)
Xy['sin_doy']=np.sin(Xy['doy']*np.pi/183)
Xy['cos_doy']=np.cos(Xy['doy']*np.pi/183)
Xy['sin_hod']=np.sin(Xy['hod']*np.pi/12)
Xy['cos_hod']=np.cos(Xy['hod']*np.pi/12)

In [13]:
Xy=Xy.drop(['Longitude','doy','hod'],axis=1)

In [14]:
Xy.to_hdf('../fourth_week/features.h5','w')

In [9]:
m=np.cos(Xy['Longitude']*np.pi/180)